Create a database

In [0]:
Create database niks

Create a Table in Amazon Redshift

In [0]:
CREATE TABLE sales_data (
    order_id INT,
    customer_id INT,
    order_date DATE,
    product_id VARCHAR(10),
    product_name VARCHAR(100),
    quantity INT,
    unit_price DECIMAL(10, 2),
    total_amount DECIMAL(10, 2)
);

Load Data into Amazon Redshift

In [0]:
COPY sales_data
FROM 's3://niks-glue-data/sample-data.csv'
IAM_ROLE 'arn:aws:iam::146962103229:role/RedshiftAccessRole'
CSV
DELIMITER ','
IGNOREHEADER 1;

After loading the data, inspect it to ensure it was loaded correctly.

In [0]:
SELECT * FROM sales_data LIMIT 10;

Data Wrangling (Cleaning and Transformation) - Handle Missing Data

In [0]:
SELECT 
    order_id,
    customer_id,
    order_date,
    product_id,
    product_name,
    quantity,
    unit_price,
    COALESCE(total_amount, 0) AS total_amount
FROM sales_data;

Remove Duplicates

In [0]:
SELECT DISTINCT * FROM sales_data;

Format Data Consistently

In [0]:
SELECT 
    order_id,
    customer_id,
    INITCAP(product_name) AS product_name, -- Standardize case
    TO_DATE(order_date, 'YYYY-MM-DD') AS order_date -- Ensure date format
FROM sales_data;

cast columns to the correct types. For example, if order_date is stored as a string, convert it into a DATE type

In [0]:
SELECT 
    order_id,
    customer_id,
    TO_DATE(order_date, 'YYYY-MM-DD') AS order_date,
    product_id,
    product_name,
    quantity,
    unit_price,
    total_amount
FROM sales_data;

Data Aggregation - Total Sales by Product

In [0]:
SELECT 
    product_name,
    SUM(total_amount) AS total_sales,
    COUNT(order_id) AS num_orders
FROM sales_data
GROUP BY product_name
ORDER BY total_sales DESC;

Data Aggregation - Sales per Customer

In [0]:
SELECT 
    customer_id,
    SUM(total_amount) AS total_spent,
    COUNT(DISTINCT order_id) AS num_orders
FROM sales_data
GROUP BY customer_id
ORDER BY total_spent DESC;

Sales Over Time (Yearly or Monthly)

In [0]:
SELECT 
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(total_amount) AS total_sales
FROM sales_data
GROUP BY year
ORDER BY year;

Sales by Month

In [0]:
SELECT 
    EXTRACT(YEAR FROM order_date) AS year,
    EXTRACT(MONTH FROM order_date) AS month,
    SUM(total_amount) AS total_sales
FROM sales_data
GROUP BY year, month
ORDER BY year, month;

Store Cleaned Data

In [0]:
CREATE TABLE cleaned_sales_data AS
SELECT 
    order_id,
    customer_id,
    TO_DATE(order_date, 'YYYY-MM-DD') AS order_date,
    INITCAP(product_name) AS product_name,
    quantity,
    unit_price,
    COALESCE(total_amount, 0) AS total_amount
FROM sales_data;

Query the Cleaned Data

In [0]:
SELECT 
    customer_id,
    product_name,
    total_amount
FROM cleaned_sales_data
WHERE total_amount > 50
ORDER BY total_amount DESC;